In [1]:
import grpc
import numpy as np

In [2]:
import sys
sys.path.append("/home/abhijit/Desktop/github_codes/model-server")

In [3]:
from model_server import server_pb2, server_pb2_grpc
from model_server.utils import create_tensor_proto, create_predict_request, create_array_from_proto, create_model_info_proto

In [4]:
channel = grpc.insecure_channel('localhost:5001')

In [5]:
# create a stub (client)
stub = server_pb2_grpc.ModelServerStub(channel)

In [6]:
input_array_dict = {"input1":create_tensor_proto(np.array([1.0,2.0]).astype(np.float32)),
                    "input2":create_tensor_proto(np.array([[10.0,11.0], [12.0,13.0]]).astype(np.float32)),
                    "input3":create_tensor_proto(np.array(["Hi".encode(), "Hello".encode(), "test".encode()]).astype(object))
                   }

In [7]:
input_array_dict

{'input1': tensor_shape {
   size: 2
 }
 tensor_content: "\000\000\200?\000\000\000@", 'input2': tensor_shape {
   size: 2
 }
 tensor_shape {
   size: 2
 }
 tensor_content: "\000\000 A\000\0000A\000\000@A\000\000PA", 'input3': tensor_shape {
   size: 3
 }
 string_val: "Hi"
 string_val: "Hello"
 string_val: "test"
 dtype: OBJECT}

In [8]:
predict_request= create_predict_request(input_array_dict, name="simple_call")

In [9]:
predict_request

name: "simple_call"
inputs {
  key: "input1"
  value {
    tensor_shape {
      size: 2
    }
    tensor_content: "\000\000\200?\000\000\000@"
  }
}
inputs {
  key: "input2"
  value {
    tensor_shape {
      size: 2
    }
    tensor_shape {
      size: 2
    }
    tensor_content: "\000\000 A\000\0000A\000\000@A\000\000PA"
  }
}
inputs {
  key: "input3"
  value {
    tensor_shape {
      size: 3
    }
    string_val: "Hi"
    string_val: "Hello"
    string_val: "test"
    dtype: OBJECT
  }
}

In [10]:
# make the call
response = stub.GetPredictions(predict_request)

In [11]:
response

name: "simple_call"
outputs {
  key: "output_array"
  value {
    tensor_shape {
      size: 2
    }
    tensor_content: "\000\000\310B\000\000HC"
  }
}

In [12]:
create_array_from_proto(response.outputs["output_array"])

array([100., 200.], dtype=float32)

# Get model Status

In [13]:
model_info_proto = create_model_info_proto({})

In [14]:
type(model_info_proto)

model_server.protos.model_info_pb2.ModelInfo

In [15]:
response = stub.GetModelInfo(model_info_proto)

In [16]:
response

info {
  version: "1"
  name: "first_model"
  status: "up"
}
info {
  version: "2"
  name: "second_model"
  status: "down"
}